In [14]:
import pandas as pd
import requests

# Load the CSV file into a pandas dataframe
movies_df = pd.read_csv("movie_metadata.csv", nrows=2000)
movies_df = movies_df.tail(1000)
movies_df

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
1000,Color,Peter Webber,209.0,131.0,73.0,163.0,Stephen Walters,879,27667947.0,Crime|Drama|Thriller,...,392.0,English,UK,R,50000000.0,2007.0,184.0,6.2,2.35,3000
1001,Color,Rob Reiner,99.0,95.0,0.0,323.0,Tim Matheson,13000,27067160.0,Comedy|Drama|Romance,...,150.0,English,USA,R,50000000.0,1999.0,559.0,5.9,1.85,0
1002,Color,Andrew Niccol,298.0,125.0,487.0,201.0,Chandler Canterbury,430,26616999.0,Action|Adventure|Romance|Sci-Fi|Thriller,...,347.0,English,USA,PG-13,40000000.0,2013.0,329.0,5.9,2.35,43000
1003,Color,John McTiernan,131.0,98.0,323.0,511.0,Harry Connick Jr.,933,26536120.0,Action|Crime|Drama|Mystery|Thriller,...,286.0,English,Germany,R,50000000.0,2003.0,631.0,6.5,2.35,0
1004,Color,Clint Eastwood,146.0,110.0,16000.0,581.0,Anjelica Huston,16000,26199517.0,Action|Crime|Drama|Mystery|Thriller,...,260.0,English,USA,R,50000000.0,2002.0,1000.0,6.4,2.35,705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,Color,Rachel Talalay,60.0,104.0,54.0,919.0,Lori Petty,6000,4064333.0,Action|Comedy|Sci-Fi,...,173.0,English,USA,R,25000000.0,1995.0,923.0,5.2,2.35,0
1996,Color,Jeffrey W. Byrd,23.0,95.0,2.0,807.0,Loretta Devine,927,4006906.0,Comedy|Crime,...,20.0,English,USA,PG-13,15000000.0,2005.0,912.0,4.1,1.85,83
1997,Color,Fernando Meirelles,229.0,121.0,353.0,18.0,Joe Pingue,45,3073392.0,Drama|Mystery|Sci-Fi|Thriller,...,300.0,English,Canada,R,25000000.0,2008.0,30.0,6.6,1.85,0
1998,Color,Uwe Boll,143.0,92.0,892.0,492.0,Udo Kier,783,1550000.0,Action|Adventure|Fantasy|Horror,...,471.0,English,USA,R,25000000.0,2005.0,595.0,2.9,2.35,937


In [15]:
# Create an empty pandas dataframe to store the API responses
api_responses_df = pd.DataFrame()

# Loop through each movie in the dataframe
for index, row in movies_df.iterrows():
    # Extract the IMDB ID from the movie IMDB link
    imdb_id = row["movie_imdb_link"].split("/")[-2]
    
    # Make an API call to the OMDB API for this movie
    api_url = f"https://www.omdbapi.com/?i={imdb_id}&plot=full&apikey=8952601b"
    response = requests.get(api_url)
    
    # Convert the JSON response to a pandas dataframe row
    movie_data = response.json()
    movie_row = pd.DataFrame.from_dict(movie_data, orient="index").T
    
    # Add the new row to the api_responses_df dataframe
    api_responses_df = pd.concat([api_responses_df, movie_row])
    
# Reset the index of the api_responses_df dataframe
api_responses_df = api_responses_df.reset_index(drop=True)

# Merge the api_responses_df dataframe with the original movies_df dataframe
merged_df = pd.concat([movies_df, api_responses_df], axis=1)

# Save the merged dataframe to a new CSV file
merged_df.to_csv("merged_movie_data2.csv", index=False)


In [16]:
api_responses_df

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response,totalSeasons
0,Hannibal Rising,2007,R,09 Feb 2007,121 min,"Adventure, Crime, Drama",Peter Webber,Thomas Harris,"Gaspard Ulliel, Rhys Ifans, Gong Li","Mischa and Hannibal, baby brother and sister, ...",...,6.1,"112,206",tt0367959,movie,06 Feb 2007,"$27,670,986",N/A,N/A,True,NaN
1,The Story of Us,1999,R,15 Oct 1999,95 min,"Comedy, Drama, Romance",Rob Reiner,"Alan Zweibel, Jessie Nelson","Bruce Willis, Michelle Pfeiffer, Colleen Rennison",Ben and Katie married fifteen years ago and gi...,...,5.9,"24,309",tt0160916,movie,01 Jun 2004,"$27,100,031","Universal Pictures, MCA/Universal Pictures [us...",N/A,True,NaN
2,The Host,2013,PG-13,29 Mar 2013,125 min,"Action, Adventure, Drama",Andrew Niccol,"Andrew Niccol, Stephenie Meyer","Saoirse Ronan, Max Irons, Jake Abel",A body-snatching alien takes possession of a y...,...,5.8,"116,404",tt1517260,movie,09 Jul 2013,"$26,627,201",N/A,N/A,True,NaN
3,Basic,2003,R,28 Mar 2003,98 min,"Action, Crime, Drama",John McTiernan,James Vanderbilt,"John Travolta, Samuel L. Jackson, Connie Nielsen","Tom Hardy, an ex-Army Ranger turned DEA agent,...",...,6.4,"64,190",tt0264395,movie,08 Jul 2003,"$26,793,311",N/A,N/A,True,NaN
4,Blood Work,2002,R,09 Aug 2002,110 min,"Action, Crime, Drama",Clint Eastwood,"Michael Connelly, Brian Helgeland","Clint Eastwood, Jeff Daniels, Anjelica Huston",Retired F.B.I. profiler Terry McCaleb (Clint E...,...,6.4,"46,055",tt0309377,movie,27 Dec 2002,"$26,235,081",N/A,N/A,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Tank Girl,1995,R,31 Mar 1995,104 min,"Action, Comedy, Sci-Fi",Rachel Talalay,"Alan Martin, Jamie Hewlett, Tedi Sarafian","Lori Petty, Ice-T, Naomi Watts",Tank Girl (Rebecca) and her friends are the on...,...,5.3,"32,888",tt0114614,movie,02 Apr 2002,"$4,064,495",N/A,N/A,True,NaN
996,King's Ransom,2005,PG-13,22 Apr 2005,95 min,"Comedy, Crime",Jeffrey W. Byrd,Wayne Conley,"Anthony Anderson, Kellita Smith, Jay Mohr",Malcolm King is a wealthy and arrogant busines...,...,4.4,"3,841",tt0388183,movie,26 Jul 2005,"$4,008,527",New Line Cinema,N/A,True,NaN
997,Blindness,2008,R,03 Oct 2008,121 min,"Drama, Mystery, Thriller",Fernando Meirelles,"José Saramago, Don McKellar","Julianne Moore, Mark Ruffalo, Gael García Bernal","A city is ravaged by an epidemic of instant ""w...",...,6.5,"73,758",tt0861689,movie,10 Feb 2009,"$3,351,751",N/A,N/A,True,NaN
998,BloodRayne,2005,R,06 Jan 2006,95 min,"Action, Fantasy, Horror",Uwe Boll,Guinevere Turner,"Kristanna Loken, Ben Kingsley, Michelle Rodriguez","In the Eighteenth Century, Rayne is the half-h...",...,2.9,"36,137",tt0383222,movie,23 May 2006,"$2,405,420",N/A,N/A,True,NaN


In [20]:
merged_df_success_response = merged_df[merged_df["Response"] == "True"]
merged_df_success_response.to_csv("merged_movie_data_success2.csv", index=False)
merged_df_success_response

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response,totalSeasons
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.1,"112,206",tt0367959,movie,06 Feb 2007,"$27,670,986",N/A,N/A,True,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.9,"24,309",tt0160916,movie,01 Jun 2004,"$27,100,031","Universal Pictures, MCA/Universal Pictures [us...",N/A,True,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.8,"116,404",tt1517260,movie,09 Jul 2013,"$26,627,201",N/A,N/A,True,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.4,"64,190",tt0264395,movie,08 Jul 2003,"$26,793,311",N/A,N/A,True,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.4,"46,055",tt0309377,movie,27 Dec 2002,"$26,235,081",N/A,N/A,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.3,"32,888",tt0114614,movie,02 Apr 2002,"$4,064,495",N/A,N/A,True,NaN
996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.4,"3,841",tt0388183,movie,26 Jul 2005,"$4,008,527",New Line Cinema,N/A,True,NaN
997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.5,"73,758",tt0861689,movie,10 Feb 2009,"$3,351,751",N/A,N/A,True,NaN
998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.9,"36,137",tt0383222,movie,23 May 2006,"$2,405,420",N/A,N/A,True,NaN
